In [63]:
#!/usr/bin/env python
import requests
import pandas as pd
import json

ip_ranges = requests.get('https://ip-ranges.amazonaws.com/ip-ranges.json').json()['prefixes']
amazon_ips = [item['ip_prefix'] for item in ip_ranges if item["service"] == "AMAZON"]
cidr = [item['ip_prefix'] for item in ip_ranges if item["service"] == "EC2"]

In [57]:
import csv
from netaddr import IPAddress 
from netaddr import IPNetwork
for networks in cidr:
    ip = IPNetwork(networks)
    subnets = list(ip.subnet(24))
    #print subnets
    count = 0
    
    while count < len(subnets):
        #print(subnets[count])
        #count +=1
        with open(r'/Users/johnsmith/notebooks/awsCidrs.csv', "ab") as f:
            writer = csv.writer(f)
            #writer = csv.writer(open("docidrs.csv", "wb"),delimiter='\n')
            writer.writerow([subnets[count]])
            #print subnets[count]
            count +=1

In [64]:
awsData = pd.read_csv(r'/Users/johnsmith/notebooks/awsCidrs.csv',names="A")
awsCidr = awsData.loc[:,'A'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
#awsCidr

In [65]:
from parsebrologs import ParseBroLogs
data = ParseBroLogs('conn.log')
df = pd.DataFrame(json.loads(data.to_json()))
#df
#server = df[~df['id.resp_h'].str.contains(':')]
#addr = server['id.resp_h']
cidr = df.loc[:,'id.resp_h'].str.rsplit(pat='.',n=1,expand=True)[0] # < -- Count by /24 CIDR block
#cidr.value_counts().head(50)

In [66]:
joined_ips=pd.merge(df,awsData, right_on=awsCidr, left_on=cidr)
joined_df = joined_ips.drop_duplicates()
#joined_df
joined_df.groupby(['id.orig_h','key_0']).size()

id.orig_h      key_0    
192.168.1.105  64.252.81    49
dtype: int64